In [85]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append('../src')

from data import news_articles, tone

%aimport data.news_articles
%aimport data.tone

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
import requests
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

In [ ]:
# mentions, # of articles, # sentiment of those articles, # tone of articles

# Fetch News Articles

In [13]:
relevant_candidates = [
    'Klobuchar',
    'Bloomberg',
    'Buttigieg',
    'Biden',
    'Warren',
    'Sanders',
]

In [43]:
article_data = news_articles.fetch_for_candidates(relevant_candidates)
df_articles = pd.DataFrame.from_records(article_data)

# Fetch Full Content

In [56]:
article_urls = df_articles['url'].values

In [67]:
def track_full_text_fetch(urls):
    count = 0
    full_text_records = []
    for url in urls:
        count += 1
        print(count)
        full_text_records.append(news_articles.fetch_full_text(url))
    return full_text_records

In [ ]:
article_text_records = track_full_text_fetch(article_urls)

In [72]:
full_text = pd.DataFrame.from_records(article_text_records)

# Combine

In [75]:
articles_full = df_articles.merge(full_text, on='url')

# Sentiment

In [79]:
analyzer = SentimentIntensityAnalyzer()
desc_sent = articles_full.description.map(lambda d: analyzer.polarity_scores(d)['compound'])
full_text_sent = articles_full.text.map(lambda d: analyzer.polarity_scores(d)['compound'])

In [82]:
articles_full['desc_sentiment'] = desc_sent
articles_full['full_text_sentiment'] = full_text_sent

# Tone

## Fetch

In [ ]:
desc_tone = articles_full.description.map(tone.get)

In [ ]:
full_text_tone = articles_full.text.map(tone.get)

In [95]:
articles_full['raw_desc_tone'] = desc_tone
articles_full['raw_text_tone'] = full_text_tone

## Proccess

In [136]:
for tone in ['analytical', 'anger', 'confident', 'fear', 'joy', 'sadness', 'tentative']:
    articles_full['desc_' + tone] = articles_full.raw_desc_tone.map(lambda d: tone.expan(d, tone))
    articles_full['full_text_' + tone] = articles_full.raw_text_tone.map(lambda d: process_document_tone(d, tone))

# Persist

In [138]:
articles_full.to_csv('../data/raw/candidate_articles.csv')